In [1]:
import os
from PIL import Image
import glob
from helpers.image_preprocessing import transform
from torchvision import models
import torch
import lmdb
from tqdm import tqdm
import pickle

In [2]:
image_dir = 'D:/header-image'
image_files = glob.glob(os.path.join(image_dir, '*.jpg'))

In [3]:
image_features = {}

resnet = models.resnet34()
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1]).to('cuda')
feature_extractor.eval()

for i, image_file in enumerate(tqdm(image_files)):
    image_id = os.path.basename(image_file).split('.')[0]
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to('cuda')

    with torch.no_grad():
        features = feature_extractor(image_tensor)
        features = features.squeeze().cpu().numpy()

    image_features[image_id] = features

100%|██████████| 97378/97378 [08:29<00:00, 190.99it/s]


In [4]:
feature_dir = 'D:/image-features'
map_size = 512 * 1024 * 1024
env = lmdb.open(feature_dir, map_size=map_size)

In [5]:
with env.begin(write=True) as txn:
        for img_id, features in tqdm(image_features.items()):
            txn.put(str(img_id).encode(), features.tobytes())

100%|██████████| 97378/97378 [00:00<00:00, 437739.60it/s]
